In [ ]:
import urllib.request, urllib.error, urllib.parse

url = 'http://tobib.spline.de/sheep/data/10000'

response = urllib.request.urlopen(url)
webContent = response.read()

print(webContent)

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8889/
TypeError: request to http://localhost:8889/api/contents/?1582505284428 failed, reason: connect ECONNREFUSED 127.0.0.1:8889

In [ ]:
%pwd

Error: Failed to connect to remote Jupyter notebook.
Check that the Jupyter Server URI setting has a valid running server specified.
http://localhost:8889/
TypeError: request to http://localhost:8889/api/contents/?1582505317189 failed, reason: connect ECONNREFUSED 127.0.0.1:8889